In [ ]:
# Script para ordenar un DataFrame de Pandas por categorías y promedios
import pandas as pd

# Cargar el archivo Excel
file_path = "escalafon_xlsx/Escalafón Masculino - Mayo.xlsx"
df = pd.read_excel(file_path, header=None)

# Identificar encabezado (fila 1 con "JUGADOR") y comenzar desde ahí
header_row_index = df[df.iloc[:, 1] == "JUGADOR"].index[0]
headers = df.iloc[header_row_index].tolist()

# Datos a partir de la fila siguiente al encabezado
data = df.iloc[header_row_index + 1:].reset_index(drop=True)

# Identificar el índice de la columna PROMEDIO
prom_col_index = headers.index("PROMEDIO")

# Identificar el índice de la columna HCP
hcp_col_index = headers.index("HCP 180")

# Procesar bloques por categoría
sorted_rows = [headers]
current_block = []
current_category_row = None

for _, row in data.iterrows():
    if isinstance(row[1], str) and "CATEGORIA" in row[1].upper():
        # Si hay un bloque acumulado, ordenarlo y agregarlo
        if current_category_row is not None and current_block:
            current_block.sort(key=lambda r: float(r[prom_col_index]) if pd.notna(r[prom_col_index]) else 0, reverse=True)
            for i, r in enumerate(current_block, 1):
                r[0] = i  # asignar posición
                sorted_rows.append(r)

        # Registrar nueva categoría
        current_category_row = row.tolist()
        sorted_rows.append(current_category_row)
        current_block = []
    elif pd.notna(row[1]):
        current_block.append(row.tolist())

# Procesar el último bloque
if current_category_row is not None and current_block:
    current_block.sort(key=lambda r: float(r[prom_col_index]) if pd.notna(r[prom_col_index]) else 0, reverse=True)
    for i, r in enumerate(current_block, 1):
        r[0] = i
        sorted_rows.append(r)

# Crear DataFrame ordenado
df_ordenado = pd.DataFrame(sorted_rows)
df_ordenado.head(20)

# Convertir la columna de promedio y HCP a tipo numérico
df_ordenado[prom_col_index] = pd.to_numeric(df_ordenado[prom_col_index], errors='coerce')
df_ordenado[hcp_col_index] = pd.to_numeric(df_ordenado[hcp_col_index], errors='coerce')

# Mantener dos decimales en la columna de promedio
df_ordenado[prom_col_index] = df_ordenado[prom_col_index].apply(lambda x: f"{x:.2f}" if pd.notna(x) else x)

# Quitar los decimales de la columna HCP
df_ordenado[hcp_col_index] = df_ordenado[hcp_col_index].apply(lambda x: f"{int(x)}" if pd.notna(x) else x)

# Guardar el DataFrame ordenado en un nuevo archivo Excel
output_file_path = "escalafon_xlsx/Escalafón Masculino - Mayo Ordenado.xlsx"
df_ordenado.to_excel(output_file_path, index=False, header=False)
print(f"Archivo ordenado guardado en: {output_file_path}")

Archivo ordenado guardado en: escalafon_xlsx/Escalafón Masculino - Mayo Ordenado.xlsx
